In [81]:
from datetime import datetime, timedelta
from random import randint, random
from typing import List, Dict

import pandas as pd

from feast import FeatureStore

# Updating the Feature Store

In [89]:
def generate_dummy_data(size: int) -> List[Dict]:
    """ Generate dummy data """
    
    data = [
        {
            "user_id": i + 1, 
            "conv_rate": round(random(), 2),
            "acc_rate": round(random(), 2),
            "avg_daily_trips": randint(1, 10),
            "event_timestamp": (datetime.now() - timedelta(days=randint(1, 30),
                                                           hours=randint(1, 24),
                                                           minutes=randint(1, 60))),
            "created": (datetime.now() - timedelta(days=randint(1, 30),
                                                           hours=randint(1, 24),
                                                           minutes=randint(1, 60))),            
        } for i in range(size)
    ]
    
    return data

In [135]:
FILE_SOURCE = "../stores/staging/data/user_hourly_purchases.parquet"

df = pd.DataFrame(
        generate_dummy_data(size=100)
    )

df.to_parquet(FILE_SOURCE, index=False)

df

,user_id,conv_rate,acc_rate,avg_daily_trips,event_timestamp,created
0,1,0.92,0.43,2,2021-10-05 20:15:42.879982,2021-09-27 07:33:42.880000
1,2,0.05,0.87,4,2021-09-28 13:13:42.880011,2021-09-20 21:13:42.880017
2,3,0.95,0.11,5,2021-10-01 06:14:42.880026,2021-10-05 07:09:42.880032
3,4,0.33,0.06,3,2021-10-04 01:22:42.880041,2021-10-08 01:35:42.880047
4,5,0.22,0.32,9,2021-09-23 18:27:42.880055,2021-10-02 14:49:42.880061
...,...,...,...,...,...,...
95,96,0.93,0.30,4,2021-09-28 01:32:42.881355,2021-10-14 23:48:42.881360
96,97,0.48,0.36,9,2021-09-22 14:28:42.881368,2021-09-19 19:41:42.881374
97,98,0.48,0.46,5,2021-09-21 18:55:42.881382,2021-09-19 07:10:42.881387
98,99,0.42,0.15,2,2021-10-07 07:39:42.881395,2021-09-25 19:11:42.881400


In [126]:
STORE = "../stores/staging"
VIEW_NAME = "user_hourly_purchases"

!cd {STORE} && feast materialize-incremental {datetime.now().isoformat()} --views {VIEW_NAME}

Materializing 1 feature views to 2021-10-19 20:11:05-04:00 into the sqlite online store.

user_hourly_purchases from 2021-12-31 19:00:00-05:00 to 2021-10-19 20:11:05-04:00:
0it [00:00, ?it/s]


# Retrieving Features from Feature Store

In [140]:
FEATURE_STORE_CONFIG = {
    "repo_path": "../stores/staging"
}

features = [
    "user_hourly_purchases:conv_rate",
    "user_hourly_purchases:acc_rate",
]

fs = FeatureStore(**FEATURE_STORE_CONFIG)

In [141]:
user_ids = (1, 2, 3)

online_features = fs.get_online_features(
                        features=features,
                        entity_rows=[{"user_id": u} for u in user_ids]
                    ).to_dict()

In [142]:
features = pd.DataFrame(online_features)

features

,acc_rate,conv_rate,user_id
0,0.43,0.92,1
1,0.87,0.05,2
2,0.11,0.95,3


# Generating Predictions

In [144]:
from random import randint

class DummyModel:
    """ Dummy Model """
    
    def predict(self, row):
        return randint(0, 1)    

In [162]:
model = DummyModel()
features["predictions"] = features.apply(model.predict)

features

,acc_rate,conv_rate,user_id,predictions
0,0.43,0.92,1,1
1,0.87,0.05,2,1
2,0.11,0.95,3,1
